In [1]:
# En notebooks/analisis_datos.ipynb

import pandas as pd

In [2]:
# Definir la ruta de los datos procesados
PROCESSED_PATH = '../data/processed/'

# Cargar el DataFrame final (Recorridos + Usuarios)
df_analisis = pd.read_csv(PROCESSED_PATH + 'recorridos_usuarios_3meses.csv')

In [3]:
# Asegurarse de que 'edad_usuario' sea numérico para el filtrado
df_analisis['edad_usuario'] = pd.to_numeric(df_analisis['edad_usuario'], errors='coerce')

print(f"DataFrame cargado. Filas totales: {len(df_analisis):,}")

DataFrame cargado. Filas totales: 831,984


#Pregunta 1: Viajes de usuarios entre 25 y 35 años

In [13]:
## --- 4.1. Análisis: Cantidad de Viajes por Rango de Edad (25 a 35 años) ---

# 1. Definir el rango de edad (rango inclusivo: 25 <= edad <= 35)
edad_min = 25
edad_max = 35

# 2. Crear la máscara de filtrado
filtro_edad = (df_analisis['edad_usuario'] >= edad_min) & (df_analisis['edad_usuario'] <= edad_max)

# 3. Aplicar el filtro y contar el número de viajes
viajes_rango_edad = df_analisis[filtro_edad].shape[0]

print(f"El total de viajes realizados por usuarios de entre {edad_min} y {edad_max} años fue:")
print(f"{viajes_rango_edad:,} viajes")

El total de viajes realizados por usuarios de entre 25 y 35 años fue:
82,464 viajes


# Pregunta 2: Top 5 estaciones de Inicio

In [19]:
## --- 4.2. Análisis: Top 5 Estaciones de Inicio ---

# 1. Agrupar el DataFrame por el nombre de la estación de origen y contar los viajes
# Contamos cuántas veces aparece cada 'nombre_estacion_origen'
conteo_estaciones = df_analisis.groupby('nombre_estacion_origen').size()

# 2. Ordenar el resultado de forma descendente y seleccionar las 5 primeras
top_5_estaciones = conteo_estaciones.sort_values(ascending=False).head(5)

# 3. Mostrar el resultado
print("\n--- Top 5 Estaciones de Inicio con mayor cantidad de viajes (Jul-Sep 2024) ---")
print(top_5_estaciones.to_string(header=False))


--- Top 5 Estaciones de Inicio con mayor cantidad de viajes (Jul-Sep 2024) ---
368 - PLAZA DE LA SHOÁ       9401
014 - Pacifico               9098
005 - Plaza Italia           8392
147 - Constitución           7881
001 - FACULTAD DE DERECHO    7718


#  Distribución por Género de Usuarios Frecuentes ---

In [38]:
# 1. Identificar usuarios frecuentes (más de 15 viajes)
# Agrupar por 'id_usuario' y contar la cantidad de viajes por usuario
conteo_viajes_usuario = df_analisis.groupby('id_usuario').size()

# Filtrar solo aquellos usuarios con más de 15 viajes
usuarios_frecuentes_id = conteo_viajes_usuario[conteo_viajes_usuario > 15].index

# 2. Filtrar el DataFrame principal para incluir solo los viajes de estos usuarios frecuentes
df_frecuentes = df_analisis[df_analisis['id_usuario'].isin(usuarios_frecuentes_id)]


# 3. Analizar la distribución por género entre estos viajes
# Contar la cantidad de viajes por género y calcular el porcentaje
distribucion_genero = df_frecuentes['genero_usuario'].value_counts()

distribucion_porcentaje = df_frecuentes['genero_usuario'].value_counts(normalize=True) * 100


print("\n--- Distribución por Género de Usuarios Frecuentes (más de 15 viajes) ---")
print("Conteo Absoluto:")
print(distribucion_genero)

print("\nDistribución Porcentual:")
print(distribucion_porcentaje.round(2).to_string(header=False) + ' %')


--- Distribución por Género de Usuarios Frecuentes (más de 15 viajes) ---
Conteo Absoluto:
genero_usuario
MALE      102597
FEMALE     45655
OTHER       1878
Name: count, dtype: int64

Distribución Porcentual:
MALE      68.34
FEMALE    30.41
OTHER      1.25 %


In [ ]:
## --- 5. Visualización: Distribución por Género ---

import matplotlib.pyplot as plt
import seaborn as sns

# 1. Preparar los datos de distribución porcentual (del paso anterior)
# Recarga la variable si es necesario, o usa la 'distribucion_porcentaje' calculada previamente
# Si reiniciaste el kernel, recalcula:
# distribucion_genero = df_frecuentes['genero_usuario'].value_counts(normalize=True) * 100 
# ... (debes asegurarte que la variable existe o recalcularla)

# Usaremos un gráfico de barras para claridad
plt.figure(figsize=(8, 6))
sns.barplot(
    x=distribucion_porcentaje.index,
    y=distribucion_porcentaje.values,
    hue=distribucion_porcentaje.index, # <-- ¡NUEVO! Asigna 'x' a 'hue'
    palette="viridis",
    legend=False                     # <-- ¡NUEVO! Oculta la leyenda que no se necesita
)

# Añadir etiquetas y título
plt.title('Distribución Porcentual de Viajes de Usuarios Frecuentes por Género (Jul-Sep 2024)')
plt.xlabel('Género del Usuario')
plt.ylabel('Porcentaje de Viajes (%)')
plt.ylim(0, 100) # Establecer el eje Y hasta 100%

# Añadir el valor porcentual encima de cada barra
for i, valor in enumerate(distribucion_porcentaje.values):
    plt.text(i, valor + 1, f'{valor:.2f}%', ha='center', va='bottom')

# 2. Guardar el gráfico en reports/figures/
FIGURES_PATH = '../reports/figures/'
nombre_archivo = 'viajes_por_genero.png'

# Crear la carpeta si no existe (importante si no la creaste con el mkdir -p inicial)
import os
if not os.path.exists(FIGURES_PATH):
    os.makedirs(FIGURES_PATH)

plt.savefig(FIGURES_PATH + nombre_archivo)
plt.close() # Cierra la figura para no mostrarla si no es necesario

print(f"\nGráfico guardado exitosamente en {FIGURES_PATH}{nombre_archivo}")


Gráfico guardado exitosamente en ../reports/figures/viajes_por_genero.png


C:\Users\user\AppData\Local\Temp\ipykernel_14736\325574077.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=distribucion_porcentaje.index, y=distribucion_porcentaje.values, palette="viridis")
